# Multivariate HMM with dimensionality reduction via dynamic factor model

### 1. necessary imports

In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from hmmlearn.hmm import GaussianHMM

%matplotlib inline
sns.set_style("darkgrid")

### 2. load data

In [3]:
Z = pd.read_csv("data/macro_panel_standardized.csv", index_col=0, parse_dates=True)
dates = Z.index
Z

,gdp_yoy,inf_yoy,rate_3m_d1,rate_10y_d1,unemployment_d1
date,,,,,
1954-06-30,-2.263432,-1.030420,-0.410992,-0.224637,0.765015
1954-09-30,-1.557446,-1.291299,0.120248,-0.015858,0.243087
1954-12-31,-0.108478,-1.465614,0.183607,0.267484,-0.895664
1955-03-31,1.270170,-1.500869,0.281082,0.371874,-0.848216
1955-06-30,1.899856,-1.492280,0.363936,0.245115,-0.468632
...,...,...,...,...,...
2024-03-31,-0.037995,-0.115659,-0.089324,-0.642194,0.053295
2024-06-30,0.016863,-0.137507,-0.001596,0.617934,0.243087
2024-09-30,-0.112489,-0.327722,-0.386623,-1.126858,0.243087


### 3. fit 2-factor DFM
- **k_factors = 2**
- **factor_order = 1** gives each factor an AR(1) dynamic

In [10]:
mod_dfm = sm.tsa.DynamicFactor(
    Z,
    k_factors=2,
    factor_order=1,
    error_order=1,
)

res_dfm = mod_dfm.fit(maxiter=1000, disp=False)
res_dfm.summary()

C:\Users\bram\scoop\apps\python\current\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QE-DEC will be used.
  self._init_dates(dates, freq)


Dep. Variable:,"['gdp_yoy', 'inf_yoy', 'rate_3m_d1', 'rate_10y_d1', 'unemployment_d1']",No. Observations:,284
Model:,"DynamicFactor(factors=2, order=1)",Log Likelihood,-1355.010
,+ AR(1) errors,AIC,2758.020
Date:,"Sun, 18 May 2025",BIC,2845.595
Time:,16:46:45,HQIC,2793.131
Sample:,06-30-1954,,
,- 03-31-2025,,
Covariance Type:,opg,,
Ljung-Box (L1) (Q):,"7.93, 48.57, 1.70, 0.27, 0.18",Jarque-Bera (JB):,"1510.44, 77.43, 3087.03, 165.58, 107845.38"
Prob(Q):,"0.00, 0.00, 0.19, 0.60, 0.67",Prob(JB):,"0.00, 0.00, 0.00, 0.00, 0.00"
Heteroskedasticity (H):,"1.28, 1.86, 0.53, 2.64, 9.25",Skew:,"-0.06, 0.04, 0.07, -0.29, 5.88"


### 4. extract smoothed factors

In [5]:
factors = res_dfm.factors.smoothed
F = pd.DataFrame(
    factors.T,
    index=dates,
    columns=["Factor1", "Factor2"]
)
F.head()

,Factor1,Factor2
date,,
1954-06-30,-1.799800,3.254212
1954-09-30,-1.137799,2.312419
1954-12-31,0.162320,0.336852
1955-03-31,1.213536,-1.678048
1955-06-30,1.580473,-2.680712


### 5. check variance explained by factors

In [11]:
common_var = (res_dfm.fittedvalues.var()).mean()
total_var = Z.var().mean()

print(f"common component explains ~= {common_var/total_var: 0.1%}")

common component explains ~=  36.5%


### 6. grid search regime count same as previous two models

In [12]:
results = []
for k in range(1, 5):
    hmm = GaussianHMM(
        n_components=k,
        covariance_type="full",
        n_iter=1000,
        random_state=42,
        verbose=False
    )

    hmm.fit(F.values)
    logL = hmm.score(F.values)
    bic = hmm.bic(F.values)
    aic = hmm.aic(F.values)
    results.append((k, logL, bic, aic, hmm))

    print(f"k={k}  logL={logL:,.1f}  AIC={aic:,.1f}  BIC={bic:,.1f}")

best_k, _, _, _, best_hmm = min(results, key=lambda t: t[2])
print(f"\nSelected k = {best_k}")


k=1  logL=-954.2  AIC=1,918.3  BIC=1,936.5
k=2  logL=-895.3  AIC=1,816.7  BIC=1,864.1
k=3  logL=-894.8  AIC=1,835.6  BIC=1,919.6
k=4  logL=-811.8  AIC=1,693.7  BIC=1,821.4

Selected k = 4
